# Course Project — UCI Credit Default (Simple)

This notebook follows the **course project guidelines** (Parts A–E) using the dataset `UCI_Credit_Card.csv`. It includes:
- Data understanding + basic stats + plots (Part A)
- PCA from scratch + sklearn comparison (Part B)
- Linear Regression with Gradient Descent from scratch (Part C)
- Logistic Regression from scratch (extra, helps for classification)
- Statistical evaluation: metrics, t-test vs baseline, bootstrap CI (Part E)

> Note: The original guideline also asks for **Perceptron + MLP**. If you want, I can add them in the same style later.


## 0) Imports and Load Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA as SKPCA
from sklearn.linear_model import LinearRegression as SKLinearRegression
from sklearn.linear_model import LogisticRegression as SKLogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

# Load dataset
df = pd.read_csv("UCI_Credit_Card.csv")
df.shape, df.columns


## A) Data Understanding & Preprocessing (15%)
- Describe dataset
- Handle missing/outliers
- Normalize features
- Compute mean/variance/covariance/correlation
- 2–3 plots + short notes

In [ ]:
# Quick look
display(df.head())
print("Shape:", df.shape)
print(df["default.payment.next.month"].value_counts(normalize=True).rename("class_ratio"))


### A1) Clean obvious category issues (simple)
In this dataset, some category codes are unusual (common in UCI version):
- EDUCATION: values like 0, 5, 6 can be grouped into 'other'
- MARRIAGE: value 0 can be grouped into 'other'

In [ ]:
df_clean = df.copy()

# EDUCATION: {0,5,6} -> 4 (other). Keep 1..4
df_clean["EDUCATION"] = df_clean["EDUCATION"].replace({0:4, 5:4, 6:4})

# MARRIAGE: {0} -> 3 (other). Keep 1..3
df_clean["MARRIAGE"] = df_clean["MARRIAGE"].replace({0:3})

# Missing values?
print("Missing values per column (top 10):")
print(df_clean.isna().sum().sort_values(ascending=False).head(10))


### A2) Handle outliers (winsorization)
Simple method: clip each numeric column to the 1st and 99th percentiles.
This reduces extreme values but keeps all rows.

In [ ]:
def winsorize_df(dataframe, cols, lower_q=0.01, upper_q=0.99):
    dfw = dataframe.copy()
    for c in cols:
        lo, hi = dfw[c].quantile(lower_q), dfw[c].quantile(upper_q)
        dfw[c] = dfw[c].clip(lo, hi)
    return dfw

numeric_cols = [c for c in df_clean.columns if c not in ["ID"]]
df_w = winsorize_df(df_clean, numeric_cols, 0.01, 0.99)


### A3) Basic statistics: mean, variance, covariance, correlation

In [ ]:
# We'll compute stats on the features only (exclude ID and target)
target_col = "default.payment.next.month"
feature_cols = [c for c in df_w.columns if c not in ["ID", target_col]]

X_all = df_w[feature_cols].values.astype(float)

mean_vec = X_all.mean(axis=0)
var_vec  = X_all.var(axis=0, ddof=1)  # sample variance

cov_mat = np.cov(X_all, rowvar=False)  # covariance matrix
corr_mat = np.corrcoef(X_all, rowvar=False)  # correlation matrix

print("Mean vector shape:", mean_vec.shape)
print("Cov matrix shape:", cov_mat.shape)
print("Corr matrix shape:", corr_mat.shape)


### A4) Visualizations (2–3 plots)
- Histogram of target (class balance)
- Histogram of LIMIT_BAL
- Scatter: AGE vs LIMIT_BAL

In [ ]:
# 1) Target histogram (bar)
counts = df_w[target_col].value_counts().sort_index()
plt.figure()
plt.bar(counts.index.astype(str), counts.values)
plt.title("Target distribution (0=No default, 1=Default)")
plt.xlabel("Class")
plt.ylabel("Count")
plt.show()

# 2) Histogram of LIMIT_BAL
plt.figure()
plt.hist(df_w["LIMIT_BAL"].values, bins=50)
plt.title("Histogram: LIMIT_BAL (Credit Limit)")
plt.xlabel("LIMIT_BAL")
plt.ylabel("Frequency")
plt.show()

# 3) Scatter AGE vs LIMIT_BAL
plt.figure()
plt.scatter(df_w["AGE"].values, df_w["LIMIT_BAL"].values, s=5, alpha=0.3)
plt.title("Scatter: AGE vs LIMIT_BAL")
plt.xlabel("AGE")
plt.ylabel("LIMIT_BAL")
plt.show()


## B) PCA from Scratch (15%)
- Implement PCA using eigen decomposition
- Scree plot
- Project to 2D and scatter plot
- Compare with sklearn PCA

In [ ]:
def pca_from_scratch(X, k=None):
    """Return (components, explained_variance_ratio, X_projected)"""
    # Center
    Xc = X - X.mean(axis=0, keepdims=True)
    # Covariance
    C = (Xc.T @ Xc) / (Xc.shape[0] - 1)
    # Eigen decomposition (C is symmetric)
    eigvals, eigvecs = np.linalg.eigh(C)   # ascending
    order = np.argsort(eigvals)[::-1]      # descending
    eigvals = eigvals[order]
    eigvecs = eigvecs[:, order]
    # Explained variance ratio
    evr = eigvals / eigvals.sum()
    if k is None:
        k = X.shape[1]
    W = eigvecs[:, :k]         # top-k eigenvectors
    X_proj = Xc @ W
    return W, evr, X_proj

# Standardize features before PCA
scaler_pca = StandardScaler()
X_scaled = scaler_pca.fit_transform(df_w[feature_cols].values.astype(float))

W, evr, X2 = pca_from_scratch(X_scaled, k=2)

# Scree plot (first 15 components)
plt.figure()
plt.plot(np.arange(1, min(16, len(evr))+1), evr[:15], marker="o")
plt.title("Scree Plot (Explained Variance Ratio) — Scratch PCA")
plt.xlabel("Component")
plt.ylabel("Explained variance ratio")
plt.show()

# 2D projection scatter
plt.figure()
plt.scatter(X2[:,0], X2[:,1], s=5, alpha=0.3)
plt.title("2D Projection — Scratch PCA (k=2)")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

# Compare with sklearn PCA (k=2)
skpca = SKPCA(n_components=2, random_state=42)
X2_sk = skpca.fit_transform(X_scaled)

print("Scratch EVR (first 2):", evr[:2])
print("sklearn EVR (first 2):", skpca.explained_variance_ratio_)


## C) Linear Regression with Gradient Descent (20%)
We need a **regression target** (continuous). The dataset’s main target is classification (default).
So for Part C, we’ll predict a continuous column:
- **y_reg = BILL_AMT1** (first bill amount)
This lets us compute MSE/MAE/R² and compare with sklearn LinearRegression.

Deliverables:
- Loss vs epochs (for different learning rates)
- Coefficient comparison vs sklearn

In [ ]:
# Prepare regression data
y_reg = df_w["BILL_AMT1"].values.astype(float)

X_reg = df_w[feature_cols].values.astype(float)  # use same features (excluding ID/target)
Xr_train, Xr_test, yr_train, yr_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

# Scale features for GD stability
scaler_reg = StandardScaler()
Xr_train_s = scaler_reg.fit_transform(Xr_train)
Xr_test_s  = scaler_reg.transform(Xr_test)

def add_bias(X):
    return np.c_[np.ones((X.shape[0], 1)), X]

Xr_train_b = add_bias(Xr_train_s)
Xr_test_b  = add_bias(Xr_test_s)

class LinearRegressionGD:
    def __init__(self, lr=0.01, epochs=500):
        self.lr = lr
        self.epochs = epochs
        self.theta = None
        self.loss_history = []

    def fit(self, Xb, y):
        n, d = Xb.shape
        self.theta = np.zeros(d)
        for _ in range(self.epochs):
            yhat = Xb @ self.theta
            error = yhat - y
            # MSE loss
            loss = (error**2).mean()
            self.loss_history.append(loss)
            # Gradient
            grad = (2/n) * (Xb.T @ error)
            self.theta -= self.lr * grad
        return self

    def predict(self, Xb):
        return Xb @ self.theta

# Train with different learning rates
lrs = [1e-4, 5e-4, 1e-3]
models = []
for lr in lrs:
    m = LinearRegressionGD(lr=lr, epochs=400).fit(Xr_train_b, yr_train)
    models.append(m)

plt.figure()
for lr, m in zip(lrs, models):
    plt.plot(m.loss_history, label=f"lr={lr}")
plt.title("Linear Regression (GD) — Loss vs Epochs")
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.legend()
plt.show()

# Pick best (lowest final loss)
best_idx = int(np.argmin([m.loss_history[-1] for m in models]))
best_lr = lrs[best_idx]
lin_gd = models[best_idx]
print("Best learning rate:", best_lr)

# Evaluate
yr_pred = lin_gd.predict(Xr_test_b)
print("Scratch Linear Regression:")
print("MSE:", mean_squared_error(yr_test, yr_pred))
print("MAE:", mean_absolute_error(yr_test, yr_pred))
print("R2 :", r2_score(yr_test, yr_pred))

# Compare with sklearn LinearRegression
sk_lin = SKLinearRegression()
sk_lin.fit(Xr_train_s, yr_train)
yr_pred_sk = sk_lin.predict(Xr_test_s)

print("\nSklearn LinearRegression:")
print("MSE:", mean_squared_error(yr_test, yr_pred_sk))
print("MAE:", mean_absolute_error(yr_test, yr_pred_sk))
print("R2 :", r2_score(yr_test, yr_pred_sk))

# Coefficient comparison (first 10)
print("\nCoefficient comparison (first 10 features):")
print("Scratch theta (bias + first 9):", lin_gd.theta[:10])
print("Sklearn coef  (first 10):      ", np.r_[sk_lin.intercept_, sk_lin.coef_[:9]])


## Logistic Regression from Scratch (Simple)
This targets the real label: `default.payment.next.month` (binary classification).
We implement logistic regression with gradient descent.

Deliverables:
- Confusion matrix
- Accuracy / Precision / Recall
- Compare to sklearn LogisticRegression

In [ ]:
# Prepare classification data
y_cls = df_w[target_col].values.astype(int)
X_cls = df_w[feature_cols].values.astype(float)

Xc_train, Xc_test, yc_train, yc_test = train_test_split(X_cls, y_cls, test_size=0.2, random_state=42, stratify=y_cls)

# Scale features
scaler_cls = StandardScaler()
Xc_train_s = scaler_cls.fit_transform(Xc_train)
Xc_test_s  = scaler_cls.transform(Xc_test)

Xc_train_b = add_bias(Xc_train_s)
Xc_test_b  = add_bias(Xc_test_s)

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

class LogisticRegressionGD:
    def __init__(self, lr=0.05, epochs=800, reg_lambda=0.0):
        self.lr = lr
        self.epochs = epochs
        self.reg_lambda = reg_lambda
        self.theta = None
        self.loss_history = []

    def fit(self, Xb, y):
        n, d = Xb.shape
        self.theta = np.zeros(d)
        for _ in range(self.epochs):
            z = Xb @ self.theta
            p = sigmoid(z)
            # Logistic loss (with small epsilon for stability)
            eps = 1e-12
            loss = -(y*np.log(p+eps) + (1-y)*np.log(1-p+eps)).mean()
            # L2 regularization (exclude bias)
            if self.reg_lambda > 0:
                loss += (self.reg_lambda/(2*n)) * np.sum(self.theta[1:]**2)
            self.loss_history.append(loss)

            # Gradient
            grad = (1/n) * (Xb.T @ (p - y))
            if self.reg_lambda > 0:
                reg = (self.reg_lambda/n) * self.theta
                reg[0] = 0.0
                grad += reg
            self.theta -= self.lr * grad
        return self

    def predict_proba(self, Xb):
        return sigmoid(Xb @ self.theta)

    def predict(self, Xb, threshold=0.5):
        return (self.predict_proba(Xb) >= threshold).astype(int)

log_gd = LogisticRegressionGD(lr=0.05, epochs=800, reg_lambda=0.1).fit(Xc_train_b, yc_train)

plt.figure()
plt.plot(log_gd.loss_history)
plt.title("Logistic Regression (GD) — Loss vs Epochs")
plt.xlabel("Epoch")
plt.ylabel("Log Loss")
plt.show()

yc_pred = log_gd.predict(Xc_test_b, threshold=0.5)

print("Scratch Logistic Regression:")
print("Accuracy :", accuracy_score(yc_test, yc_pred))
print("Precision:", precision_score(yc_test, yc_pred, zero_division=0))
print("Recall   :", recall_score(yc_test, yc_pred, zero_division=0))
print("Confusion matrix:\n", confusion_matrix(yc_test, yc_pred))

# Compare with sklearn LogisticRegression
sk_log = SKLogisticRegression(max_iter=2000)
sk_log.fit(Xc_train_s, yc_train)
yc_pred_sk = sk_log.predict(Xc_test_s)

print("\nSklearn LogisticRegression:")
print("Accuracy :", accuracy_score(yc_test, yc_pred_sk))
print("Precision:", precision_score(yc_test, yc_pred_sk, zero_division=0))
print("Recall   :", recall_score(yc_test, yc_pred_sk, zero_division=0))
print("Confusion matrix:\n", confusion_matrix(yc_test, yc_pred_sk))


## E) Statistical Evaluation (15%)
Guidelines ask for:
- Regression: MSE, MAE, R² ✅ (done above)
- Classification: Accuracy, Precision, Recall ✅ (done above)
- **t-test comparing model to baseline**
- **Bootstrap confidence interval for accuracy**

We’ll do:
1) Baseline classifier: predict the majority class from training.
2) 5-fold CV on a **subset** for speed.
3) Paired t-test on fold accuracies.
4) Bootstrap 95% CI on test accuracy.

In [ ]:
# --- 1) Baseline (majority class) on the test set ---
majority_class = int(pd.Series(yc_train).mode()[0])
baseline_pred = np.full_like(yc_test, majority_class)
acc_baseline_test = accuracy_score(yc_test, baseline_pred)
acc_model_test = accuracy_score(yc_test, yc_pred)

print("Test Accuracy — Baseline:", acc_baseline_test)
print("Test Accuracy — Scratch Logistic:", acc_model_test)

# --- 2) 5-fold CV (subset for speed) ---
def kfold_indices(n, k=5, seed=42):
    rng = np.random.default_rng(seed)
    idx = np.arange(n)
    rng.shuffle(idx)
    folds = np.array_split(idx, k)
    return folds

# Use a smaller subset for quick CV
n_sub = 6000
sub_idx = np.random.default_rng(42).choice(len(X_cls), size=n_sub, replace=False)
X_sub = X_cls[sub_idx]
y_sub = y_cls[sub_idx]

# scale inside each fold (important)
folds = kfold_indices(n_sub, k=5, seed=42)

acc_model_folds = []
acc_base_folds = []

for i in range(5):
    test_idx = folds[i]
    train_idx = np.hstack([folds[j] for j in range(5) if j != i])

    Xtr, ytr = X_sub[train_idx], y_sub[train_idx]
    Xte, yte = X_sub[test_idx], y_sub[test_idx]

    sc = StandardScaler()
    Xtr_s = sc.fit_transform(Xtr)
    Xte_s = sc.transform(Xte)

    Xtr_b = add_bias(Xtr_s)
    Xte_b = add_bias(Xte_s)

    # Baseline majority on this fold
    maj = int(pd.Series(ytr).mode()[0])
    y_base = np.full_like(yte, maj)
    acc_base_folds.append(accuracy_score(yte, y_base))

    # Train scratch logistic quickly (fewer epochs for CV)
    m = LogisticRegressionGD(lr=0.05, epochs=400, reg_lambda=0.1).fit(Xtr_b, ytr)
    yhat = m.predict(Xte_b)
    acc_model_folds.append(accuracy_score(yte, yhat))

acc_model_folds = np.array(acc_model_folds)
acc_base_folds = np.array(acc_base_folds)

print("Fold Accuracies (model):   ", acc_model_folds)
print("Fold Accuracies (baseline):", acc_base_folds)
print("Mean model acc:", acc_model_folds.mean(), " | Mean baseline acc:", acc_base_folds.mean())

# --- 3) Paired t-test (if SciPy exists) ---
try:
    from scipy import stats
    t_stat, p_val = stats.ttest_rel(acc_model_folds, acc_base_folds)
    print("Paired t-test: t =", t_stat, ", p =", p_val)
except Exception as e:
    print("SciPy not available for t-test. Error:", e)
    # simple fallback: just print the differences
    diffs = acc_model_folds - acc_base_folds
    print("Differences per fold:", diffs, " | mean diff:", diffs.mean())

# --- 4) Bootstrap 95% CI for test accuracy ---
rng = np.random.default_rng(42)
B = 2000
acc_samples = []
n_test = len(yc_test)
for _ in range(B):
    bs_idx = rng.integers(0, n_test, size=n_test)
    acc_samples.append(accuracy_score(yc_test[bs_idx], yc_pred[bs_idx]))
acc_samples = np.array(acc_samples)
ci_low, ci_high = np.percentile(acc_samples, [2.5, 97.5])
print(f"Bootstrap 95% CI for Scratch Logistic test accuracy: [{ci_low:.4f}, {ci_high:.4f}]")


## Summary Table (Quick)
This is a small summary you can copy to your report.

In [ ]:
summary = pd.DataFrame({
    "Task": ["Regression (BILL_AMT1)", "Classification (Default)"],
    "Model": ["Linear Regression (GD scratch)", "Logistic Regression (GD scratch)"],
    "Main Metrics": [
        f"MSE={mean_squared_error(yr_test, yr_pred):.2f}, MAE={mean_absolute_error(yr_test, yr_pred):.2f}, R2={r2_score(yr_test, yr_pred):.4f}",
        f"Acc={accuracy_score(yc_test, yc_pred):.4f}, Prec={precision_score(yc_test, yc_pred, zero_division=0):.4f}, Rec={recall_score(yc_test, yc_pred, zero_division=0):.4f}"
    ]
})
display(summary)
